## Prueba tecnica Ingeniero de Datos Pragma
Presentado por: Nicolas Restrepo Carvajal

* Ingeniero de Datos, MsC en Ingeniería , Docente universitario

In [1]:
import sqlite3
import os
import time 
import csv
import os
import time 
import pandas as pd


In [6]:

def connect_sqlite(path):
    conn = sqlite3.connect(path)
    #Creando el cursor
    cursor = conn.cursor()
    return conn
    
def close_sqlite(conn):
    conn.close()

def execute(statement):
    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()
    query = statement
    cursor.execute(statement)
    conn.commit()
    rows = cursor.fetchall()

    for row in rows:
        print(row)
    conn.close()

   

def insert_csv_line_sqlite(directory_path,sqlite_db_path,table_name):
    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()
    files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    datasets = {}
    
    for file in files:
        file_path = os.path.join(directory_path, file)
        dataset_name = os.path.splitext(file)[0]
        print(file_path)
        print(dataset_name)
        
        with open(file_path, newline='', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)
            for row in csv_reader:
                query = f'INSERT INTO {table_name} (timestamp, price, user_id) VALUES (?, ?, ?)'
                cursor.execute(query, row)
        conn.commit()
    conn.close()
            
            
    return datasets

In [7]:

if __name__ == "__main__":
    raw_data_path = "../Data/raw"
    sqlite_db_path='../database/pragma.db'
    table_name='prices'
    create_table = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        timestamp DATE,
        price double precision,
        user_id INTEGER
    );
    """
    execute(create_table)
    insert_csv_line_sqlite(raw_data_path,sqlite_db_path,table_name)


../Data/raw/2012-5.csv
2012-5
../Data/raw/2012-1.csv
2012-1
../Data/raw/2012-3.csv
2012-3
../Data/raw/2012-4.csv
2012-4
../Data/raw/2012-2.csv
2012-2


In [8]:
select_table=f"""select * from {table_name} order by date(timestamp)"""
execute(select_table)

(0, '2/1/2012', 76, 1)
(1, '2/2/2012', 66, 1)
(2, '2/3/2012', 47, 10)
(3, '2/4/2012', 100, 5)
(4, '2/5/2012', 19, 4)
(5, '2/6/2012', 59, 8)
(6, '2/7/2012', 77, 3)
(7, '2/8/2012', 100, 8)
(8, '2/9/2012', 55, 1)
(9, '2/10/2012', 19, 2)
(10, '2/11/2012', 95, 3)
(11, '2/12/2012', 73, 2)
(12, '2/13/2012', 32, 1)
(13, '2/14/2012', 95, 5)
(14, '2/15/2012', 12, 8)
(15, '2/16/2012', 37, 8)
(16, '2/17/2012', 28, 10)
(17, '2/18/2012', 10, 5)
(18, '2/19/2012', 27, 9)
(19, '2/20/2012', 14, 9)
(20, '2/21/2012', 87, 5)
(21, '2/22/2012', 75, 7)
(22, '2/23/2012', 36, 8)
(23, '2/24/2012', 84, 8)
(24, '2/25/2012', 62, 6)
(25, '2/26/2012', 56, 6)
(26, '2/27/2012', 17, 4)
(27, '2/28/2012', 64, 7)
(28, '2/29/2012', 68, 3)
(None, '5/1/2012', 82, 7)
(None, '5/2/2012', 61, 1)
(None, '5/3/2012', 19, 3)
(None, '5/4/2012', 94, 3)
(None, '5/5/2012', 17, 5)
(None, '5/6/2012', 19, 3)
(None, '5/7/2012', 66, 6)
(None, '5/8/2012', 13, 7)
(None, '5/9/2012', 31, 6)
(None, '5/10/2012', 95, 6)
(None, '5/11/2012', 77, 7)
(N